In [ ]:
import os

import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression

from utils.func import read_data
from utils.metric import evaluate, eval_pope

In [ ]:
model_name = "LLaVA-7B"
prompt = "oe"

train_data, x_train, y_train = read_data(model_name, "POPE", split="train", 
                                prompt=prompt, token_idx=0, return_data=True)
val_data, x_val, y_val = read_data(model_name, "POPE", split="val", 
                                   prompt=prompt, token_idx=0, return_data=True)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

### The original performance of LVLMs

In [ ]:
for i in range(len(val_data)):
    val_data[i]['pred'] = 1 if val_data[i]['response'].lower().startswith('yes') else 0
    
for category in ["adversarial", "popular", "random"]:
    print(category)
    
    label_list = [ins['label'] for ins in val_data
                  if ins['category'] == category]
    pred_list = [ins['pred'] for ins in val_data
                  if ins['category'] == category]
    
    eval_pope(label_list, pred_list)

### Linear probing

In [ ]:
# Logits
print(x_train.shape, x_val.shape)

model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict_proba(x_val)[:, 1]
evaluate(y_val, y_pred, show=True)

In [ ]:
for i in range(len(val_data)):
    val_data[i]['pred'] = 1 if y_pred[i] > 0.5 else 0
    
for category in ["adversarial", "popular", "random"]:
    print(category)
    label_list = [ins['label'] for ins in val_data
                  if ins['category'] == category]
    pred_list = [ins['pred'] for ins in val_data
                  if ins['category'] == category]
    eval_pope(label_list, pred_list)